# Analyse Run #

This is a template notebook to browse the results of a evolution simulation.

Please _Restart & Run All_ to make shure you start with a clean notebook.

## Import required libraries

In [10]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from  ipywidgets import widgets 
from ipywidgets import interact, interactive, fixed
from IPython.display import display,HTML,clear_output
import os
HTML('''<script>code_show=true;function code_toggle() {if (code_show){$('div.input').hide();} else {$('div.input').show();} code_show = !code_show} $( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')
import phievo.AnalysisTools as AT
from  phievo.AnalysisTools.Notebook import Notebook
notebook = Notebook()

## Select the Project

In [2]:
notebook.select_project.display()

## Select Seed

In [3]:
notebook.select_seed.display()

## Plot observable

In [4]:
notebook.plot_evolution_observable.display()

## Select Generation

In [5]:
notebook.select_generation.display()

## PLot Layout

In [6]:
notebook.plot_layout.display()

## Run Dynamics

In [7]:
notebook.run_dynamics.display()

## Plot Dynamics

In [8]:
notebook.plot_dynamics.display()

## Plot Cell Profile

In [9]:
notebook.plot_cell_profile.display()